In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [5]:
#Load the csv exported in Part I to a DataFrame
df = pd.read_csv('city_data_copy.csv')
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ambon City,-3.6954,128.1814,78.94,83,20,1.14,ID,1654640184
1,Arrondissement de Troyes,48.3333,4.0833,57.25,88,0,5.75,FR,1654640185
2,Ushuaia,-54.8000,-68.3000,44.26,57,20,12.66,AR,1654640185
3,George Town,5.4112,100.3354,80.53,83,20,0.00,MY,1654639916
4,Faanui,-16.4833,-151.7500,79.52,76,25,12.93,PF,1654640185


In [11]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    df[['Lat', 'Lng']], weights=df['Humidity'])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
#create dataframe of ideal places to visit based on desired weather 
ideal_temp_df = df.loc[df['Max Temp'] < 80]
ideal_temp_df = ideal_df.loc[ideal_df['Max Temp'] >= 74]
ideal_temp_df.describe()
ideal_wind_df = ideal_temp_df[ideal_temp_df['Wind Speed'] < 10]
ideal_cloud_df = ideal_wind_df[ideal_wind_df['Cloudiness'] == 0]
hotel_df = ideal_cloud_df.dropna(axis=0, how='any')
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
198,Yangi Marg`ilon,40.4272,71.7189,74.12,38,0,9.22,UZ,1654640241,
230,Ponta do Sol,32.6667,-17.1000,74.03,83,0,2.39,PT,1654640251,
298,Pop,40.8736,71.1089,75.69,33,0,3.56,UZ,1654640271,
389,Sirte,31.2089,16.5887,76.15,70,0,5.77,LY,1654640298,
416,Chardara,41.2547,67.9692,77.45,31,0,5.48,KZ,1654640305,
461,Arrondissement d'Ajaccio,42.0000,8.9167,75.25,73,0,2.30,FR,1654640285,
547,Kiryat Gat,31.6100,34.7642,77.40,75,0,2.73,IL,1654640343,


In [51]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"rankby": "distance",
    "type": "hotel",
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    city_name = row['City']
    city_lat = row['Lat'] 
    city_lng = row['Lng']
    city_lat = str(city_lat)
    city_lng = str(city_lng)
    location = city_lat + ',' + city_lng
    params['location'] = location
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")

Retrieving Results for Index 198: Yangi Marg`ilon.
Closest hotel to Yangi Marg`ilon is Mashal Koʻchasi 42A.
------------
Retrieving Results for Index 230: Ponta do Sol.
Closest hotel to Ponta do Sol is Cais da Ponta do Sol.
------------
Retrieving Results for Index 298: Pop.
Closest hotel to Pop is Dustlik Kuchasi.
------------
Retrieving Results for Index 389: Sirte.
Closest hotel to Sirte is السجل المدني سرت.
------------
Retrieving Results for Index 416: Chardara.
Closest hotel to Chardara is Infolife Shardara.
------------
Retrieving Results for Index 461: Arrondissement d'Ajaccio.
Closest hotel to Arrondissement d'Ajaccio is Residence avec Services Olivier Bleu Noel Sarrola.
------------
Retrieving Results for Index 547: Kiryat Gat.
Closest hotel to Kiryat Gat is בית כנסת.
------------


In [52]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
198,Yangi Marg`ilon,40.4272,71.7189,74.12,38,0,9.22,UZ,1654640241,Mashal Koʻchasi 42A
230,Ponta do Sol,32.6667,-17.1000,74.03,83,0,2.39,PT,1654640251,Cais da Ponta do Sol
298,Pop,40.8736,71.1089,75.69,33,0,3.56,UZ,1654640271,Dustlik Kuchasi
389,Sirte,31.2089,16.5887,76.15,70,0,5.77,LY,1654640298,السجل المدني سرت
416,Chardara,41.2547,67.9692,77.45,31,0,5.48,KZ,1654640305,Infolife Shardara
461,Arrondissement d'Ajaccio,42.0000,8.9167,75.25,73,0,2.30,FR,1654640285,Residence avec Services Olivier Bleu Noel Sarrola
547,Kiryat Gat,31.6100,34.7642,77.40,75,0,2.73,IL,1654640343,בית כנסת


In [53]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [60]:
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info)

fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))